In [1]:
#Packages:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from camera_matrix import camera_matrix
import time
import seaborn as sns
import matplotlib.patches as mpatches
from skimage.segmentation import felzenszwalb

In [15]:
def segment_water(image):
  """
  Segments the image to identify potential water bodies.

  Args:
      image: A NumPy array representing the RGB image.

  Returns:
      A NumPy array with the same dimensions as the input image,
      where each pixel value represents the segmented class (water or non-water).
  """

  # Convert the image to HSV color space
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

  # Define a threshold range for blue color (water) in HSV
  lower_blue = np.array([50, 100, 50])  # Adjust these values as needed
  upper_blue = np.array([255, 140, 255])  # Adjust these values as needed

  # Create a mask for the blue color range
  mask = cv2.inRange(hsv, lower_blue, upper_blue)

  # Apply morphological operations (optional)
  # You can experiment with opening or closing operations to refine the mask
  kernel = np.ones((5, 5), np.uint8)
  mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
  mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

  # Segment the image based on the mask
  segmented_image = np.zeros_like(image)
  segmented_image[mask > 0] = (0, 0, 255)  # Assign red color to water pixels

  return segmented_image

video_capture = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/Design-Project/vid1_dx.MP4")
# Lire la première frame de la vidéo
ret, first_frame = video_capture.read()

# Example usage (replace 'path/to/image.jpg' with your image path)
image = first_frame
segmented_image = segment_water(image)

# Display the original and segmented image
cv2.imshow('Original Image', image)
cv2.imshow('Segmented Image (Water in Red)', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

: 

In [11]:
# Charger la vidéo
video_capture = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/Design-Project/vid1_dx.MP4")
#video_capture = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/GX010031.MP4")
#video_capture = cv2.VideoCapture("/Users/nilsmanni/Desktop/MA2/design_project/GX010056.MP4")
# Définition de la fonction NDVI
def ndvi(image):
    # Convertir l'image en float
    image = image.astype(float)
    # Calculer l'indice NDVI
    ndvi = (image[:,:,2] - image[:,:,0]) / (image[:,:,2] + image[:,:,0] + 1e-5)
    return ndvi

# Définition de la fonction NDWI
def ndwi(image):
    # Convertir l'image en float
    image = image.astype(float)
    # Calculer l'indice NDWI
    ndwi = (image[:,:,1] - image[:,:,0]) / (image[:,:,1] + image[:,:,0] + 1e-5)
    return ndwi

# Lire la première frame de la vidéo
ret, first_frame = video_capture.read()

# Vérifier si la première frame a été lue avec succès
if not ret:
    print("Impossible de lire la première frame de la vidéo.")
    exit()

# Segmentation de la première frame en utilisant l'algorithme de Felzenszwalb
segments = felzenszwalb(first_frame, scale=100, sigma=0.5, min_size=50)

# Calculer l'indice NDVI et NDWI
ndvi_image = ndvi(first_frame)
ndwi_image = ndwi(first_frame)

# Seuils pour la détection de la rivière et de la berge
ndvi_threshold = 0.1  # Seuil NDVI pour la berge
ndwi_threshold = 0.1  # Seuil NDWI pour la rivière

# Créer des masques pour la rivière et la berge en utilisant les indices
river_mask = np.zeros_like(first_frame)
shore_mask = np.zeros_like(first_frame)

river_mask[ndwi_image > ndwi_threshold] = [0, 0, 255]  # Rouge pour la rivière
shore_mask[ndvi_image > ndvi_threshold] = [0, 255, 0]  # Vert pour la berge

# Appliquer une opération de morphologie pour lisser les zones
river_kernel = np.ones((15, 15), np.uint8)  # Noyau pour la rivière
shore_kernel = np.ones((15, 15), np.uint8)  # Noyau pour la berge

river_mask = cv2.morphologyEx(river_mask, cv2.MORPH_OPEN, river_kernel)
shore_mask = cv2.morphologyEx(shore_mask, cv2.MORPH_OPEN, shore_kernel)

# Fusionner les masques
result = cv2.addWeighted(river_mask, 0.8, shore_mask, 0.8, 0)
# Afficher le résultat
#cv2.imshow('River Image', result)
cv2.imshow('River Image', shore_mask)
# Attendre jusqu'à ce que l'utilisateur appuie sur une touche
cv2.waitKey(0)

# Libérer la vidéo et fermer les fenêtres
video_capture.release()
cv2.destroyAllWindows()